# AI Governance Policy Analysis

This notebook demonstrates how to use the AI Governance Tool for interactive policy analysis and visualization.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from ai_governance_tool import ComplianceAnalyzer

# Initialize analyzer
analyzer = ComplianceAnalyzer()

## Load and Analyze Policies

In [ ]:
def analyze_policy(content: str) -> dict:
    """Analyze a single policy and return results as dict."""
    result = analyzer.check_compliance(content)
    
    # Convert to dictionary
    return {
        'overall_score': result.score,
        'status': 'PASS' if result.is_compliant else 'FAIL',
        'category_scores': result.category_scores,
        'patterns': result.found_patterns,
        'proximity': result.proximity_scores
    }

# Load example policies
policies_dir = Path('../sample_policies')
results = []

for policy_file in policies_dir.glob('*.md'):
    with open(policy_file) as f:
        content = f.read()
        
    result = analyze_policy(content)
    result['policy'] = policy_file.stem
    results.append(result)

# Convert to DataFrame
df = pd.DataFrame(results)

## Visualize Results

In [ ]:
# Set style
plt.style.use('seaborn')

# Create figure with multiple subplots
fig = plt.figure(figsize=(15, 10))

# 1. Overall scores
plt.subplot(2, 2, 1)
sns.barplot(data=df, x='policy', y='overall_score')
plt.title('Overall Compliance Scores')
plt.xticks(rotation=45)

# 2. Category scores heatmap
plt.subplot(2, 2, 2)
category_scores = pd.DataFrame([r['category_scores'] for r in results])
sns.heatmap(category_scores, annot=True, cmap='RdYlGn')
plt.title('Category Scores Heatmap')

# 3. Pattern matches
plt.subplot(2, 2, 3)
pattern_counts = df['patterns'].apply(lambda x: sum(len(v) for v in x.values()))
sns.histplot(pattern_counts)
plt.title('Pattern Match Distribution')

# 4. Pass/Fail distribution
plt.subplot(2, 2, 4)
status_counts = df['status'].value_counts()
plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%')
plt.title('Compliance Status Distribution')

plt.tight_layout()
plt.show()

## Pattern Proximity Analysis

In [ ]:
# Create proximity matrix
def create_proximity_matrix(results):
    # Get all unique patterns
    all_patterns = set()
    for result in results:
        for scores in result['proximity'].items():
            all_patterns.update(scores[0].split(' - '))
    
    patterns = sorted(all_patterns)
    n = len(patterns)
    matrix = np.zeros((n, n))
    
    # Fill matrix with average proximity scores
    for i, p1 in enumerate(patterns):
        for j, p2 in enumerate(patterns):
            if i == j:
                matrix[i,j] = 1.0
                continue
                
            # Look for proximity score
            scores = []
            for result in results:
                key = f"{p1} - {p2}"
                rev_key = f"{p2} - {p1}"
                if key in result['proximity']:
                    scores.append(result['proximity'][key])
                elif rev_key in result['proximity']:
                    scores.append(result['proximity'][rev_key])
            
            if scores:
                matrix[i,j] = np.mean(scores)
    
    return pd.DataFrame(matrix, index=patterns, columns=patterns)

# Create and plot proximity matrix
proximity_df = create_proximity_matrix(results)

plt.figure(figsize=(12, 10))
sns.heatmap(proximity_df, annot=True, cmap='viridis')
plt.title('Pattern Proximity Matrix')
plt.show()

## Generate Summary Report

In [ ]:
# Create summary report
summary = pd.DataFrame({
    'Total Policies': len(df),
    'Pass Rate': (df['status'] == 'PASS').mean() * 100,
    'Average Score': df['overall_score'].mean(),
    'Min Score': df['overall_score'].min(),
    'Max Score': df['overall_score'].max()
}, index=[0])

print("Summary Statistics:")
display(summary)

# Export detailed results
with pd.ExcelWriter('policy_analysis_results.xlsx') as writer:
    df.to_excel(writer, sheet_name='Overall Results', index=False)
    category_scores.to_excel(writer, sheet_name='Category Scores')
    proximity_df.to_excel(writer, sheet_name='Pattern Proximity')
    summary.to_excel(writer, sheet_name='Summary', index=False)